## 1. RawData 가져오기

In [1]:
import glob
import os
import re

txt_file_path = 'data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Whatever you want to do', 'Is all right with me']


## 2. 데이터 전처리

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

corpus = []

for sentence in raw_corpus:
    size = len(sentence)
    # 한 문장의 단어 개수가 15개 이상일 경우 제외
    if len(sentence) == 0 or len(sentence.split()) > 15: continue

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

print(corpus[:10])

['<start> let s stay together i , i m i m so in love with you <end>', '<start> whatever you want to do <end>', '<start> is all right with me <end>', '<start> cause you make me feel so brand new <end>', '<start> loving you forever <end>', '<start> is what i need <end>', '<start> let me , be the one you come running to <end>', '<start> i ll never be untrue oh baby <end>', '<start> let s , let s stay together gether <end>', '<start> lovin you whether , whether <end>']


## 3. 텐서플로우를 사용하여 토큰 생성

In [3]:
import tensorflow as tf

def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')


    # print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]
print(src_input.shape, tgt_input.shape)
print(src_input[:3])

(168590, 32) (168590, 32)
[[  2  63  16 223 281   5   4   5  22   5  22  30  14  37  31   7   3   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  2 590   7  65  10  48   3   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  2  26  25  85  31  12   3   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]]


## 4. 훈련용/테스트용 데이터 분리 (80/20)

In [4]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, random_state=42, test_size=0.2)

print(f'훈련용 데이터: {enc_train.shape}, {dec_train.shape}')
print(f'테스트용 데이터: {enc_val.shape}, {dec_val.shape}')

훈련용 데이터: (134872, 32), (134872, 32)
테스트용 데이터: (33718, 32), (33718, 32)


## 5. 모델 생성 및 훈련

In [5]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
526/526 [==============================] - 224s 397ms/step - loss: 1.7275
Epoch 2/10
526/526 [==============================] - 213s 404ms/step - loss: 1.4280
Epoch 3/10
526/526 [==============================] - 213s 404ms/step - loss: 1.3525
Epoch 4/10
526/526 [==============================] - 213s 405ms/step - loss: 1.2955
Epoch 5/10
526/526 [==============================] - 213s 406ms/step - loss: 1.2456
Epoch 6/10
526/526 [==============================] - 214s 406ms/step - loss: 1.1993
Epoch 7/10
526/526 [==============================] - 214s 406ms/step - loss: 1.1559
Epoch 8/10
526/526 [==============================] - 213s 406ms/step - loss: 1.1146
Epoch 9/10
526/526 [==============================] - 214s 406ms/step - loss: 1.0760
Epoch 10/10
526/526 [==============================] - 214s 406ms/step - loss: 1.0391


## 6. 문장 생성

In [10]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m gonna be alright <end> '

## 회고

- 어려워서 각 단계의 실행문들과 옵션을 전혀 이해하지 못했다.
- 모델 훈련을 할 경우, 아이펠 노트북에서는 에러가 나지 않지만, 로컬 또는 깃헙에서는 에러가 난다.
- 자연어 처리를 한번 공부해야겠다.